In [ ]:
import os, time, random
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, models

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


In [ ]:
CFG = {
    "images_dir": r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\Project Folder\Images_10k",
    "labels_csv": r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\Project Folder\Final Dataset\Dataset_CNN.csv",

    "img_size": 224,
    "batch_size": 64,
    "epochs": 20,
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "use_class_balancing": True,  # if unstable, set False
}

BRACKET_NAMES = {0:"0–25%", 1:"25–50%", 2:"50–75%", 3:"75–100%"}
print("Images dir exists?", Path(CFG["images_dir"]).exists())
print("Labels csv exists?", Path(CFG["labels_csv"]).exists())


In [ ]:
labels = pd.read_csv(CFG["labels_csv"])
assert {"filename","bracket"}.issubset(labels.columns), "labels.csv must have filename, bracket."

labels["path"] = labels["filename"].apply(lambda f: str(Path(CFG["images_dir"]) / f))
missing = labels.loc[~labels["path"].apply(lambda p: Path(p).exists())]
assert missing.empty, f"Missing images:\n{missing.head()}"

y = labels["bracket"].astype(int).values
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=SEED)
train_idx, val_idx = next(sss.split(labels["path"], y))
train_df, val_df = labels.iloc[train_idx].reset_index(drop=True), labels.iloc[val_idx].reset_index(drop=True)

print("Train size:", len(train_df), " Val size:", len(val_df))
print("Train class counts:", train_df["bracket"].value_counts().sort_index().to_dict())
print("Val   class counts:", val_df["bracket"].value_counts().sort_index().to_dict())


In [ ]:
tfm_train = transforms.Compose([
    transforms.Resize((CFG["img_size"], CFG["img_size"])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15, fill=0),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
tfm_val = transforms.Compose([
    transforms.Resize((CFG["img_size"], CFG["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class MetaCells(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")
        x = self.transform(img)
        y = int(row["bracket"])
        return x, y

train_ds = MetaCells(train_df, tfm_train)
val_ds   = MetaCells(val_df,   tfm_val)


In [ ]:
if CFG["use_class_balancing"]:
    counts = train_df["bracket"].value_counts().sort_index().values.astype(float)
    weights = 1.0 / counts
    sample_w = train_df["bracket"].map({i:w for i,w in enumerate(weights)}).values
    sampler = WeightedRandomSampler(sample_w, num_samples=len(sample_w), replacement=True)
    train_dl = DataLoader(train_ds, batch_size=CFG["batch_size"], sampler=sampler,
                          num_workers=0, pin_memory=torch.cuda.is_available())
else:
    train_dl = DataLoader(train_ds, batch_size=CFG["batch_size"], shuffle=True,
                          num_workers=0, pin_memory=torch.cuda.is_available())

val_dl   = DataLoader(val_ds,   batch_size=CFG["batch_size"], shuffle=False,
                      num_workers=0, pin_memory=torch.cuda.is_available())

print("Batches — train:", len(train_dl), " val:", len(val_dl))


In [ ]:
def make_model(num_classes=4):
    m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    m.fc = nn.Linear(m.fc.in_features, num_classes)
    return m

model = make_model(4).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG["epochs"])
criterion = nn.CrossEntropyLoss()

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()


In [ ]:
xb, yb = next(iter(train_dl))
print("Batch shapes:", xb.shape, yb.shape)
xb, yb = xb.to(device), yb.to(device)
model.train()
logits = model(xb)
print("Logits shape:", logits.shape)
loss = criterion(logits, yb)
print("Loss:", float(loss))
optimizer.zero_grad(set_to_none=True); loss.backward(); optimizer.step()
print("Smoke test OK ✅")


In [ ]:
history = {"train_acc":[], "val_acc":[]}

@torch.no_grad()
def evaluate():
    model.eval()
    n, tl, ta = 0, 0.0, 0.0
    all_p, all_t = [], []
    for xb, yb in val_dl:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        b = yb.size(0)
        n += b; tl += loss.item()*b; ta += accuracy(logits, yb)*b
        all_p.append(logits.argmax(1).cpu().numpy()); all_t.append(yb.cpu().numpy())
    return tl/n, ta/n, np.concatenate(all_p), np.concatenate(all_t)

def train():
    best = 0.0
    for e in range(1, CFG["epochs"]+1):
        model.train(); n=0; tl=0.0; ta=0.0; t0=time.time()
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb); loss = criterion(logits, yb)
            optimizer.zero_grad(set_to_none=True); loss.backward(); optimizer.step()
            b = yb.size(0); n += b; tl += loss.item()*b; ta += accuracy(logits, yb)*b
        scheduler.step()
        vl, va, _, _ = evaluate()
        history["train_acc"].append(ta/n); history["val_acc"].append(va)
        print(f"Epoch {e:02d} | train_loss {tl/n:.4f} acc {ta/n:.4f} | val_loss {vl:.4f} acc {va:.4f} | {time.time()-t0:.1f}s")
        if va > best:
            best = va
            torch.save({"model": model.state_dict()}, "best.pt")
    print("Best val acc:", best)

train()


In [ ]:
vl, va, preds, tgts = evaluate()
print(f"\nFINAL — Val loss: {vl:.4f} | Val accuracy: {va:.4f}\n")
print(classification_report(tgts, preds, target_names=[BRACKET_NAMES[i] for i in range(4)]))

cm = confusion_matrix(tgts, preds, labels=[0,1,2,3])
fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(range(4)); ax.set_yticks(range(4))
ax.set_xticklabels([BRACKET_NAMES[i] for i in range(4)], rotation=30, ha='right')
ax.set_yticklabels([BRACKET_NAMES[i] for i in range(4)])
for (i,j), v in np.ndenumerate(cm):
    ax.text(j, i, int(v), ha='center', va='center')
ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title("Confusion Matrix")
plt.show()

plt.figure()
plt.plot(history["train_acc"], label="Train Acc")
plt.plot(history["val_acc"], label="Val Acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.title("Accuracy per Epoch")
plt.show()


In [ ]:
ckpt = torch.load("best.pt", map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

infer_tf = transforms.Compose([
    transforms.Resize((CFG["img_size"], CFG["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

@torch.no_grad()
def predict_image(path):
    img = Image.open(path).convert("RGB")
    x = infer_tf(img).unsqueeze(0).to(device)
    logits = model(x)
    probs = logits.softmax(1).cpu().numpy()[0]
    k = int(np.argmax(probs))
    return {
        "pred_bracket_id": k,
        "pred_bracket_name": BRACKET_NAMES[k],
        "probs": {BRACKET_NAMES[i]: float(probs[i]) for i in range(4)}
    }


predict_image(r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\Images\unitcell_row_0123.png")
